In [2]:
from vision_explanation_methods import DRISE_runner as dr
from helpers.pytorch_yolov8_wrapper import PytorchYoloV8Wrapper
from ultralytics import YOLO

def compute_saliency_maps(results, image_location, model):
    # print(image_location)
    # print(model)
    return dr.get_drise_saliency_map(nummasks=100, imagelocation=image_location, model= PytorchYoloV8Wrapper(model), savename="anything", numclasses=80, max_figures=2, maskres=(4,4))

c:\Users\santiago.calderon\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np


def move_image(image, delta_x, delta_y, dimensions):
    height, width = image.shape[:2]
    new_image = np.zeros((dimensions[0], dimensions[1]))
    # Calculate the new position for pasting the image onto the canvas
    new_x_start, new_y_start = max(0, delta_x), max(0, delta_y)
    new_x_end, new_y_end = min(dimensions[1], width + delta_x), min(
        dimensions[0], height + delta_y
    )

    # Calculate the position to copy from the original image
    orig_x_start, orig_y_start = max(0, -delta_x), max(0, -delta_y)
    orig_x_end, orig_y_end = min(width, dimensions[1] - delta_x), min(
        height, dimensions[0] - delta_y
    )

    # Copy the image onto the new canvas at the new position
    new_image[new_y_start:new_y_end, new_x_start:new_x_end] = image[
        orig_y_start:orig_y_end, orig_x_start:orig_x_end
    ]

    return new_image



# image_size = (300, 400)
# new_image_size = (600, 800)
# color1 = 255
# color2 = 0
# delta_x = 50
# delta_y = -50
# test_pattern = np.zeros((image_size[0], image_size[1]), dtype=np.uint8)

# for y in range(image_size[0]):
#     color = color1 if y % 50 < 25 else color2
#     test_pattern[y, :] = color

# # Display the test pattern
# plt.imshow(test_pattern)
# plt.show()

# result = move_image(test_pattern, delta_x, delta_y, new_image_size)
# plt.imshow(result)
# plt.show()

In [4]:
import cv2
import matplotlib.pyplot as plt
import numpy as np


def scale_image(image, scale_x, scale_y):
    width = int(image.shape[1] * scale_x)
    height = int(image.shape[0] * scale_y)
    dim = (width, height)
    scaled_image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    return scaled_image

# image_size = (400, 300)
# color1 = (255, 255, 255)
# color2 = (0, 0, 0)

# test_pattern = np.zeros((image_size[1], image_size[0], 3), dtype=np.uint8)

# for y in range(image_size[1]):
#     color = color1 if y % 50 < 25 else color2
#     test_pattern[y, :] = color

# # Display the test pattern
# plt.imshow(test_pattern)
# plt.show()

# scaled = scale_image(test_pattern, 2, 0.5)
# plt.imshow(scaled)
# plt.show()

# scaled = scale_image(test_pattern, 0.5, 2)
# plt.imshow(scaled)
# plt.show()

In [5]:
import numpy as np

def compute_explanation(img, class_id, exp, first_level, model):
    if(first_level == -1):
        x = np.flip(np.arange(0, exp.shape[1]))
        y = np.arange(0, exp.shape[0])
        X, Y = np.meshgrid(x, y)

        levels = np.flip(np.unique(exp))
        masks = np.empty([exp.shape[0], exp.shape[1], 3])
        min_expl = []
        # print(class_id)
        for level in levels:
            pixels = np.where(exp == level)
            masks[pixels[0], pixels[1], :] = True
            min_expl = np.where(masks, img, 0)
            pre = model.predict([min_expl])
            if (class_id not in pre[0].boxes.cls.numpy()):
                continue
            break
        first_level = level
        print('Finished first explanation')
    else:
        pixels = np.where(exp >= first_level)
        masks = np.empty([exp.shape[0], exp.shape[1], 3])
        masks[pixels[0], pixels[1], :] = True
        min_expl = np.where(masks, img, 0)
    return min_expl, first_level

In [6]:
from sort import *
import cvzone
from sort import *
import math
import numpy as np
import cv2

import cv2
import matplotlib.pyplot as plt
from scripts.metrics import compute_insertion

def calculate_intersection_over_union(square1, square2):
    x1 = max(square1[0], square2[0])
    y1 = max(square1[1], square2[1])
    x2 = min(square1[2], square2[2])
    y2 = min(square1[3], square2[3])

    area_square1 = (square1[2] - square1[0]) * (square1[3] - square1[1])
    area_square2 = (square2[2] - square2[0]) * (square2[3] - square2[1])

    intersection_area = max(0, x2 - x1) * max(0, y2 - y1)

    union_area = area_square1 + area_square2 - intersection_area

    iou = intersection_area / union_area if union_area > 0 else 0

    return iou


def get_key_by_value(dictionary, value):
    max_iou = 0
    best_key = None
    for key, square in dictionary.items():
        iou = calculate_intersection_over_union(square, value)
        if iou > max_iou:
            max_iou = iou
            best_key = key
    return best_key


def plot_boxes(results, img, detections, model):
    classes = []
    confidence = {}
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            cls = int(box.cls[0])
            currentClass = model.model.names[cls]

            conf = math.ceil(box.conf[0] * 100) / 100
            currentArray = np.array([x1, y1, x2, y2, conf])
            detections = np.vstack((detections, currentArray))
            classes.append(cls)
            confidence[(x1, y1, x2, y2)] = conf
            # cvzone.putTextRect(img, f'{str(currentClass).upper()}: {conf}', (x1+10,y1-10), scale=1.5, thickness=2, colorR=(0,0,255),font=cv2.FONT_HERSHEY_PLAIN)
            # img = cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0), thickness=3)
    return [detections, classes], img, confidence


def track_detect(r, img, tracker, model, first_id, previous_dim, results, exp, first_exp, box_index, image_location, class_name, confidence_dict):
    detections, _ = r
    resultTracker = tracker.update(detections)
    center_changes = {}
    counter = 0
    height = img.shape[0]
    width = img.shape[1]
    light_blue = (31, 112, 255)
    for i, res in enumerate(resultTracker):
        x1, y1, x2,y2, id = res
        x1, y1, x2, y2, id = int(x1), int(y1), int(x2), int(y2), int(id)
        matching_index_x = 0
        if (first_id == id):
            x1_old, y1_old, x2_old, y2_old = previous_dim
            best_iou = 0
            for i, detection in enumerate(detections):
                iou = calculate_intersection_over_union((detection[0], detection[1], detection[2], detection[3]), (x1, y1, x2, y2))
                if iou > best_iou:
                    matching_box = detection
                    matching_index_x = i
            confidence = confidence_dict[(matching_box[0], matching_box[1], matching_box[2], matching_box[3])]
            print(f'x_old: {x1_old}, y_old: {y1_old}, x2_old: {x2_old}, y2_old: {y2_old}')
            old_x = abs(x1_old - x2_old)
            old_y = abs(y1_old - y2_old)
            new_x = abs(x1 - x2)
            new_y = abs(y1 - y2)

            scale_x = new_x / old_x
            scale_y = new_y / old_y
            exp_scaled = scale_image(first_exp, scale_x, scale_y)

            old_bb_center = np.array([(x1_old + x2_old) / 2, (y1_old + y2_old) / 2])
            new_bb_center = np.array([(x1 + x2) / 2, (y1 + y2) / 2])

            old_image_center = np.array([width / 2, height / 2])
            scaled_image_center = np.array([exp_scaled.shape[1] / 2, exp_scaled.shape[0] / 2])
            scaled_bb_center = np.array([(old_bb_center[0] - old_image_center[0]) * scale_x + scaled_image_center[0], (old_bb_center[1] - old_image_center[1]) * scale_y + scaled_image_center[1]])

            center_changes = new_bb_center - scaled_bb_center

            exp = move_image(
                np.transpose(exp_scaled, (0, 1)),
                int(center_changes[0]),
                int(center_changes[1]),
                (height, width),
            )
            cvzone.putTextRect(
                img,
                text=f"{class_name}:{confidence}, id: {first_id}",
                pos=(x1 + 10, y1 - 10),
                scale=1.5,
                thickness=2,
                colorR=light_blue,
                font=cv2.FONT_HERSHEY_PLAIN,
            )
            img = cv2.rectangle(img, (x1, y1), (x2, y2), light_blue, thickness=3)
            return (exp, first_exp, previous_dim, first_id, img, (x1, y1, x2, y2), matching_index_x)

        if first_id == -1 and (
            detections[box_index][0] == x1
            and detections[box_index][1] == y1
            and detections[box_index][2] == x2
            and detections[box_index][3] == y2
        ):
            print(f"x: {x1}, y: {y1}, x2: {x2}, y2: {y2}")
            first_id = id
            print(f'Class: {class_name}, Confidence: {confidence_dict[(x1,y1,x2,y2)]}')
            res = compute_saliency_maps(results, image_location, model)
            # print(image_location)
            # print(results)
            print("Finished first saliency map")
            # print(confidence_dict)
            cvzone.putTextRect(
                img,
                text=f"{class_name}: {confidence_dict[(x1,y1,x2,y2)]}, id: {first_id}",
                pos=(x1 + 10, y1 - 10),
                scale=1.5,
                thickness=2,
                colorR=light_blue,
                font=cv2.FONT_HERSHEY_PLAIN,
            )
            img = cv2.rectangle(img, (x1, y1), (x2, y2), light_blue, thickness=3)
            # print(len(res))

            return (
                res[box_index]["detection"][0].numpy(),
                res[box_index]["detection"][0].numpy(),
                (x1, y1, x2, y2),
                first_id,
                img,
                (x1, y1, x2, y2),
                box_index
            )
        counter += 1
    return ([], first_exp, previous_dim, first_id, img, previous_dim, box_index)


def track_saliency_maps(frame_number, car_number, box_index_first_frame):
    model = YOLO("yolov8n.pt")
    tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)
    first_id = -1
    previous_dim = (0, 0, 0, 0)
    exp = [0]
    first_exp = []
    frames = []
    auc_results = []
    auc_results_2 = []
    first_level = -1
    class_id = -1
    class_name = ""
    light_blue = (31, 112, 255)
    tolerance_counter = 0
    max_frame_tolerance = 30 * 5
    while True:
        image_location = f"datasets/car/car-{car_number}/img/{str(frame_number).zfill(8)}.jpg"
        if not os.path.exists(image_location):
            break
        img = cv2.imread(image_location)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = model(img, verbose=False)
        if (first_id == -1):
            plt.imshow(results[0].plot())
            plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
            plt.axis('off')
            plt.tight_layout()
            plt.show()
        detections, frame, confidence_dict = plot_boxes(results, img, np.empty((0,5)), model)
        _, classes = detections
        if len(results[0].boxes.cls) <= box_index_first_frame:
            box_index_first_frame = 0
        #print(classes)
        if class_name == "":
            if len(results[0].boxes.cls) == 0:
                break
            class_name = model.names[classes[box_index_first_frame]]
        det, classes = detections
        class_dict = {}
        for det, cl in zip(det, classes):
            class_dict[tuple(det[0:4].astype(np.int32))] = cl
        original = img.copy()
        #print(detections)
        new_exp, first_exp, previous_dim, first_id, img_boxes, current_dim, matching_index = track_detect(
                detections,
                img,
                tracker,
                model,
                first_id,
                previous_dim,
                results,
                exp,
                first_exp,
                box_index_first_frame,
                image_location,
                class_name,
                confidence_dict
            )
        
        if abs(previous_dim[0] - previous_dim[2]) == 0 or abs(previous_dim[1] - previous_dim[3]) == 0:
            print("Skipped")
            break

        if len(new_exp) == 0:
            tolerance_counter += 1
            if tolerance_counter > max_frame_tolerance:
                for i in range(max_frame_tolerance):
                    frames.pop()
                break
            frames.append(cv2.hconcat([original, np.zeros_like(original)]))
            frame_number += 1
            continue
        else:
            tolerance_counter = 0

        exp = new_exp
        viridis_frame = plt.cm.viridis(new_exp)
        viridis_frame_rgb = viridis_frame[:, :, :3]  # Extract RGB channels

        # Blending using OpenCV's addWeighted
        alpha = 0.5  # You can adjust the alpha value as needed
        dst = cv2.addWeighted(
                img_boxes, alpha, (viridis_frame_rgb * 255).astype(np.uint8), 1 - alpha, 0
            )
        if (class_id == -1):
            class_id = results[0].boxes.cls.numpy().astype("uint32")[box_index_first_frame]

        min_expl, first_level = compute_explanation(original, class_id, exp, first_level, model)

        min_expl = cv2.rectangle(
                min_expl,
                (current_dim[0], current_dim[1]),
                (current_dim[2], current_dim[3]),
                light_blue,
                thickness=3,
            )
        print(matching_index, box_index_first_frame)
        res = compute_saliency_maps(results, image_location, model)
        exp_2 = res[matching_index]["detection"][0].numpy()
    
        viridis_frame = plt.cm.viridis(exp_2)
        viridis_frame_rgb = viridis_frame[:, :, :3]  # Extract RGB channels
        alpha = 0.5  # You can adjust the alpha value as needed
        dst2 = cv2.addWeighted(
                img_boxes, alpha, (viridis_frame_rgb * 255).astype(np.uint8), 1 - alpha, 0
            )
        print(dst2.shape, dst.shape)
        final_frame = cv2.hconcat([dst, min_expl, dst2])

        frames.append(final_frame)
        frame_number += 1
        
        results = compute_insertion(model = model, saliency_map=exp, image=original, class_index=class_id)
        results_2 = compute_insertion(model = model, saliency_map=exp_2, image=original, class_index=class_id)
        auc_results.append(results)
        auc_results_2.append(results_2)
        print(f"AUC: {results}")
        print(f"AUC_2: {results_2}")
        
    return frames, auc_results, auc_results_2

In [7]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import matplotlib

matplotlib.rcParams["animation.embed_limit"] = 2**128

plt.ioff()
# Function to update each frame
def update(frame):
    plt.clf()  # Clear the previous plot
    plt.imshow(frame, cmap="viridis")  # Show the new frame
    plt.axis("off")
    plt.tight_layout()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    plt.gca().set_axis_off()

def create_video(update, frames):
    ratio = frames[0].shape[0] / frames[0].shape[1]
    constant = 8
    # Create animation
    fig = plt.figure(figsize=(constant, constant * ratio))
    ani = animation.FuncAnimation(fig, update, frames=frames, interval=60)
    # Display animation as HTML
    return HTML(ani.to_jshtml())

In [8]:
import matplotlib as mpl
mpl.rcParams["savefig.pad_inches"] = 0
import random

def save_video(frames, frame_number, car_set_object, box_index):
    FPS = 30
    out = cv2.VideoWriter(
        f"videos/{frame_number}_{car_set_object}_{box_index}.mp4",
        cv2.VideoWriter_fourcc(*"mp4v"),
        FPS,
        (frames[0].shape[1], frames[0].shape[0]),
    )
    for frame in frames:
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        out.write(frame)
    out.release()

def track_saliency_maps_with_video(frame_number, car_set_object, box_index):

    print(f"Frame number: {frame_number}, Car number: {car_set_object}, Explanation index: {box_index}")

    frames, auc_results, aux_results_2 = track_saliency_maps(frame_number=frame_number, car_number=car_set_object, box_index_first_frame=box_index)
    
    plt.boxplot(auc_results)
    plt.show()
    plt.boxplot(aux_results_2)
    plt.show()
    print(f"Number of frames: {len(frames)}")
    if len(frames) > 0:
        display(create_video(update, frames))
        save_video(frames, frame_number, car_set_object, box_index)

In [9]:
track_saliency_maps_with_video(1468, 10, 1)

# for j in range(1, 100):
#     frame_number = random.randint(1, 1500)
#     car_number = random.randint(11, 20)
#     while car_number == 7 or car_number == 5 or car_number == 4:
#         car_number = random.randint(1, 10)
#     box_index = random.randint(0, 3)
#     track_saliency_maps_with_video(frame_number, car_number, box_index)

Frame number: 1468, Car number: 10, Explanation index: 1


KeyboardInterrupt: 